# **Assignment 1 - Language Models**
#### **Due: September 21, 2023 (11:59PM)**

### **Notes**

### **Introduction**

Welcome to CSE 527A. Each assignment will contain two parts: a written and coding portion. The coding portion for each homework assignment will be delivered through a Colaboratory notebook such as this one. Please use as many code and markdown cells to run and explain all the steps you took in order to answer each question for each problem. Please keep in mind the collaboration policy as specified in the Academic Integrity section of the syllabus and cite all external sources and/or collaborators.

### **Comments/Documentation**

Please follow PEP 8 style guidelines (https://peps.python.org/pep-0008/) for commenting your code. Furthermore, please remember to manually save your work once in a while. If you are connected to a hosted runtime that if for whatever reason it disconnects you will have to rerun all connected code cells.

### **Getting Started**

In order to compile code efficiently, enable GPU support.

To access a GPU, go to `Edit->Notebook settings` and in the `Hardware accelerator` dropdown choose `GPU`.
As soon as you run a code cell, you will be connected to a cloud instance with a GPU.
Try running the code cell below to check that a GPU is connected (select the cell then either click the play button at the top left or press `Ctrl+Enter` or `Shift+Enter`).

The free version of Google Colab will provide the necessary hardware for this course. Please keep in mind the RAM and Disk Space that you are allocated and that you are not given an infinite active runtime.

### **Submission Instructions**

We will use Gradescope for assignment submission. Submit the written and code portions of the assignment to the respective assignments. **Please note if notebook output is cleared, you will receive a 0**. To download this notebook, go to `File->Download .ipynb`.

When submitting your ipython notebooks, make sure everything runs correctly if the cells are executed in order starting from a fresh session.  Note that just because a cell runs in your current session doesn't mean it doesn't rely on code that you have already changed or deleted.

Note that Gradesope will allow you to submit multiple times before the deadline, and we will use the latest submission for grading.

## **Setup**

In [ ]:
from google.colab import drive # one option to load datasets
from google.colab import files
drive.mount('/content/gdrive')
!nvidia-smi -L # check if using GPU

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
GPU 0: Tesla T4 (UUID: GPU-43e79988-d56d-5eb5-4b58-1edfcb135a55)


In [ ]:
!pip install datasets

In [ ]:
# various imports you might find useful, feel free to import additional packages
from datasets import load_dataset
import string
from functools import reduce
from heapq import nlargest
from tqdm import tqdm
import numpy as np
import re

## Problem 1: N-grams

In [ ]:
# Loading the Dataset (this is done for you)
train_split,val_split,test_split = load_dataset('wikitext', 'wikitext-2-v1',
                                                           split =['train','validation','test'])
train_df = train_split.to_pandas()
val_df = val_split.to_pandas()
test_df = test_split.to_pandas()

In [ ]:
train_df

,text
0,
1,= Valkyria Chronicles III = \n
2,
3,Senjō no Valkyria 3 : <unk> Chronicles ( Japa...
4,"The game began development in 2010 , carrying..."
...,...
36713,Common starlings may be kept as pets or as la...
36714,The common starling 's gift for mimicry has l...
36715,Mozart had a pet common starling which could ...
36716,Common starlings are trapped for food in some...


In [ ]:
# Drop rows with '' value
train_df = train_df[train_df['text']!= '']
train_df = train_df.reset_index()
train_df =  train_df.drop('index', axis=1)
train_df
# len(train_df)

,text
0,= Valkyria Chronicles III = \n
1,Senjō no Valkyria 3 : <unk> Chronicles ( Japa...
2,"The game began development in 2010 , carrying..."
3,"It met with positive sales in Japan , and was..."
4,= = Gameplay = = \n
...,...
23762,= = = In science and culture = = = \n
23763,Common starlings may be kept as pets or as la...
23764,The common starling 's gift for mimicry has l...
23765,Mozart had a pet common starling which could ...


### Problem 1.1: Write a program to compute unsmoothed unigrams, bigrams, and trigrams.

TODO: Fill in the `remove_punctuation` method below.

In [ ]:
import string
def remove_punctuation(text)-> str:
    """Given an input text, this utility function should strip punctuation,
    extra white spaces, and from unknown tokens (<unk>) from the text.

    Args:
        text (str): input text

    Returns:
        processed_text (str): processed input text
    """
    text = text.replace('<unk>', '')

    for i in text:
        if i in string.punctuation:
            text = text.replace(i, "")

    text = ' '.join(text.split())


    return text

for i in range(len(train_df)):
  train_df['text'][i] = remove_punctuation(train_df['text'][i])

TODO: Fill in the `n_gram` method below. Do not import NLTK.
>Hint: Don't forget to remove punctuation from the input text.

In [ ]:
def n_gram(text,context,ngram):
    """Method to calculate the ngrams of a specific degree from a piece of text. Unseen ngrams
    should be added to context, while already seen ngrams should have their occurance count increased.

    Args:
        text (str): unprocessed input text
        context (Dict[str, int]): Dictionary mapping each already seen ngram to it's occurance count
        ngram (int): degree of ngram to compute (i.e. compute bigrams when ngram=2)

    Returns:
        context (Dict[str, int]): Dictionary mapping each seen ngram to it's occurance count
        populated with the ngrams from the input text
    """
    text = text.split(" ")

    for i in range(len(text) - ngram + 1):
        ngram_value = ' '.join(text[i : i+ngram])
        #  if an n-gram is not present in the context, it defaults to a count of 0.
        context[ngram_value] = context.get(ngram_value, 0) + 1
    return context



### Problem 1.2: Train the model on the wikitext-2-v1 corpus

TODO: Train the model on the wikitext-2-v1 corpus by calculating all the unigrams, bigrams, and trigrams. Then, print out the top 10 most frequent unigrams, bigrams, and trigrams. Do not import NLTK.

In [ ]:
train_df

,text
0,Valkyria Chronicles III
1,Senjō no Valkyria 3 Chronicles Japanese 戦場のヴァル...
2,The game began development in 2010 carrying ov...
3,It met with positive sales in Japan and was pr...
4,Gameplay
...,...
23762,In science and culture
23763,Common starlings may be kept as pets or as lab...
23764,The common starling s gift for mimicry has lon...
23765,Mozart had a pet common starling which could s...


In [ ]:
from operator import itemgetter
N = 10


for n in range(1, 4):
    print(f"{n}-grams")
    context = {}

    for i in range(len(train_df)):
        context = n_gram(train_df['text'][i], context, n)
    top_10 = dict(sorted(context.items(), key=itemgetter(1), reverse=True)[:N])
    print(top_10)



1-grams
{'the': 113161, 'of': 56889, 'and': 50603, 'in': 39486, 'to': 39190, 'a': 34250, 'was': 20985, 'The': 17602, 's': 14322, 'that': 14135}
2-grams
{'of the': 17052, 'in the': 10606, 'to the': 6041, 'and the': 4347, 'on the': 4202, 'for the': 3519, 'from the': 2898, 'at the': 2737, 'by the': 2721, 'as a': 2668}
3-grams
{'one of the': 778, 'the United States': 655, 'as well as': 580, 'part of the': 527, 'the end of': 501, 'end of the': 359, 'in the United': 347, 'a number of': 273, 'at the time': 246, 'known as the': 245}


Write-Up: Explain the differences between your most common unigrams. Bigrams? Trigrams?


*   Unigrams captures more articles, conjunctions, prepositions, and adverbs.
*   Bigrams are pairs of adjacent words that frequently occur together. They represent two-word combinations, or "collocations".
*   Trigrams sequences of three words that occur together frequently.
Some of them seem to represent specific phrases. (e.g., one of the, the united states, and as well as.)

## Problem 2: Word Embeddings

In this problem, we'll be working with pretrained word vectors. We'll be using the package gensim to work with pretrained Word2Vec vectors.

In [ ]:
!pip install gensim -U

In [ ]:
import gensim
from gensim import downloader
wv = downloader.load("glove-wiki-gigaword-100") # loading pretrained word vectors

### Problem 2.1: Finding similar words

TODO: Complete this function which returns the cosine_similarity between two vectors.
>Hint: Recall the cosine similarty defined as follows: $$similarity(u, v) = \frac{u \cdot v}{||u||\cdot||v||}$$

In [ ]:
import numpy as np
from numpy.linalg import norm
# code reference: https://www.geeksforgeeks.org/how-to-calculate-cosine-similarity-in-python/

def cosine_similarity(u, v):
  """Function to calculate the cosine similary between two vectors

  Args:
    u (ndarray): vector
    v (ndarray): vector

  Returns:
    similarity (float): The cosine similarity between vectors u and v
  """

  # compute cosine similarity
  cosine = np.dot(u, v)/(norm(u)*norm(v))
  # print("Cosine Similarity:", cosine)

  return cosine

TODO: Implement the function below. It should return the word that is most similar to positive_words and least similar to negative words.

> Hint: You might find Gensim's documentation on [KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.wmdistance) and [Word2Vec Models](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html) helpful for syntax, but you should *not* call their most_similar method.

In [ ]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/400000 is the
word #1/400000 is ,
word #2/400000 is .
word #3/400000 is of
word #4/400000 is to
word #5/400000 is and
word #6/400000 is in
word #7/400000 is a
word #8/400000 is "
word #9/400000 is 's


In [ ]:
def most_similar(wv, input_word):
  """Function to calculate the word most similar to input_word

  Args:
    wv (KeyedVector): pretrained word vectors
    input_word (List(str)): input_word

  Returns:
    best_word, best_score (Tuple(str, float))
  """
  input_word_v = wv[input_word]
  best_word = None
  best_score = -1


  for word in wv.index_to_key:
      if word != input_word:
          word_v = wv[word]
          similarity = cosine_similarity(input_word_v, word_v)

          if similarity > best_score:
              best_word = word
              best_score = similarity

  return best_word, best_score


TODO: Using the method you wrote above, find the most similar words to: dog, whale, before, however, fabricate.

In [ ]:
print('dog\'s most similar word: ', most_similar(wv,"dog"))
print('whale\'s most similar word: ', most_similar(wv,"whale"))
print('before\'s most similar word: ', most_similar(wv,"before"))
print('however\'s most similar word: ', most_similar(wv,"however"))
print('fabricate\'s most similar word: ', most_similar(wv,"fabricate"))

dog's most similar word:  ('cat', 0.8798076)
whale's most similar word:  ('whales', 0.80388796)
before's most similar word:  ('after', 0.92460865)
however's most similar word:  ('although', 0.96575457)
fabricate's most similar word:  ('invent', 0.7040183)


### Problem 2.2: Using word embeddings to solve analogies

TODO: Implement the following function to to calculate the word most similar to positive_words and least similar to negative words. This function should be similar to the `most_similar` function above, but more robust in the sense that is should be able to handle multiple positive and negative words.
> Hint: Use vector addition and subtraction to compute target vectors for the analogies. Again, you should not call Gensim's `most_similar` method.

In [ ]:
def most_similar_robust(wv, positive_words=None, negative_words=None):
    """Function to calculate the word most similar to positive_words and least similar to negative words

    Args:
        wv (KeyedVector): pretrained word vectors
        positive_words (List[str]): List of positive words
        negative_words (List[str]): List of negative words

    Returns:
        best_word (str): The word most similar to positive_words and least similar to negative words
    """

    best_score = -99999
    best_word = None

    if positive_words is None:
        positive_words = []
    if negative_words is None:
      negative_words = []

    # formula reference from https://kawine.github.io/blog/nlp/2019/06/21/word-analogies.html
    # Compute the vector by adding vectors of positive words and subtracting vectors of negative words
    # vd = vb + (vc - va)
    word_4 = np.zeros(wv.vector_size)
    for word in positive_words:
        word_4 += wv[word]
    for word in negative_words:
        word_4 -= wv[word]

    for word in wv.index_to_key:
        if word in [positive_words[0], positive_words[1], negative_words[0]]:
            continue

        wvec = wv[word]

        similarity = cosine_similarity(wvec,word_4)
        if similarity > best_score:
            best_score = similarity
            best_word = word


    return best_word, best_score


TODO: Use the method above to solve the following analogies, where a is to b as c is to ___ “, which is often represented as a : b :: c : d :

 vd = vb + (vc - va) == vb – va = vd – vc. Find the most similar vector in the corpus to vd.
- dog:puppy :: cat:?
- man:king :: woman:?
- france:wine :: england:?

In [ ]:
analogies = [("cat", "puppy", "dog"), ("woman", "king", "man"), ("england", "wine", "france")]

for analogy in analogies:
    word_1, word_2, word_3 = analogy


    # Find the missing word using most_similar_robust function
    best_word, best_score  = most_similar_robust(wv, [word_1, word_2], [word_3])

    # Print the analogy and the missing word
    print(f"For analogy '{word_3}:{word_2}::{word_1}:?', the missing word is '{best_word}' with similarity score {best_score}'\n")



For analogy 'dog:puppy::cat:?', the missing word is 'kitten' with similarity score 0.6617627544171691'

For analogy 'man:king::woman:?', the missing word is 'queen' with similarity score 0.7834413916974592'

For analogy 'france:wine::england:?', the missing word is 'tea' with similarity score 0.6066607723493729'

